## Visión por Computadora 1

## Trabajo Práctico N° 3

### Grupo

| Autores               | E-mail                    | Nº SIU  |
|---------------------- |---------------------------|---------|
| Braian Desía          | b.desia@hotmail.com       | a1804   |
| Juan José Cardinali   | juanchijc@gmail.com       | a1809   |

#### LIBRERÍAS

In [37]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

### ENUNCIADO

Encontrar el logotipo de la gaseosa dentro de las imágenes provistas en *Material_TPs/TP3/imagesa partir del templateMaterial_TPs/TP3/template*.

1. (4puntos) Obtener una detección del logo en cada imagen sin falsos positivos.

2. (4puntos) Plantear y validar un algoritmo para múltiples detecciones en la imagen *coca_multi.pngconelmismotémplatedelítem1*.

3. (2puntos) Generalizar el algoritmo del item2 para todas las imágenes.

Visualizar los resultados con bounding boxes en cada imagen mostrando el nivel de confianza de la detección.

### RESOLUCIÓN